### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Celiac_Disease/GSE23418'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset likely contains gene expression data
is_gene_available = False  # aCGH data indicates chromosomal copy number aberrations, not gene expression data

# Determine availability of variables
sample_characteristics = {
    0: ['age: 64', 'age: 87', 'age: 66', 'age: 73', 'age: 35', 'age: 59', 'age: 55', 'age: 50', 'age: 34', 'age: 61', 'age: 29', 'age: 51', 'age: 60', 'age: 67', 'age: 80', 'age: 81', 'age: 69', 'age: 63', 'age: 52', 'age: 79', 'age: 77'], 
    1: ['sex: female', 'sex: male'], 
    2: ['tumor site: Duodenum', 'tumor site: Ileum', 'tumor site: Jejunum', 'tumor site: ILeum', 'tumor site: duodenum'], 
    3: ['celiac disease: No', 'celiac disease: Yes'], 
    4: ['tnm: pT3N1Mx', 'tnm: pT4N1Mx', 'tnm: pT3N0Mx', 'tnm: pT4NxMx', 'tnm: pT4N0M1', 'tnm: pT4N0Mx', 'tnm: pT4N1M1', 'tnm: pT1NxMx', 'tnm: pT3N1M1', 'tnm: pT3N1M0', 'tnm: pT4N1M0', 'tnm: pT3N0M0', 'tnm: pT1N0Mx', 'tnm: pT4N0M0'], 
    5: ['platform_id type: oligo']
}

# Set keys for available data
trait_row = 3 if len(set(sample_characteristics[3])) > 1 else None
age_row = 0 if len(set(sample_characteristics[0])) > 1 else None
gender_row = 1 if len(set(sample_characteristics[1])) > 1 else None

# Define conversion functions
def convert_trait(value):
    v = value.split(':')[-1].strip().lower()
    if v == 'yes':
        return 1
    elif v == 'no':
        return 0
    return None

def convert_age(value):
    try:
        return int(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    v = value.split(':')[-1].strip().lower()
    if v == 'female':
        return 0
    elif v == 'male':
        return 1
    return None

# Save metadata
save_cohort_info('GSE23418', './preprocessed/Celiac_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, extract clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Celiac_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Celiac_Disease/trait_data/GSE23418.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
